# Pipeline to cnovert national dashboards shapefiles into vector tiles

[Data sources](https://docs.google.com/document/d/1D9kGrF9AUXDrMLqm-51ZfI43vlfYqpzcQd3mk7MIQqk/edit#heading=h.z8hx51w9ls2m)


In [ ]:
import os
from pathlib import Path
import geopandas
import subprocess
import logging
import requests
import gzip
import fiona
from typing import Union

In [ ]:
#  FIXME: This will depends from where the notebook kernel is running so be careful
WORK_DIR =Path(os.getcwd())
BASE_DIR = f'{WORK_DIR.parents[3]}/work/datasets/raw'
logging.basicConfig(level=logging.INFO)

# @TODO: Add expected data files source as an environment variable. and add the download bit for the data sources.
assert BASE_DIR == '/home/jovyan/work/datasets/raw', f'{BASE_DIR} is not the correct directory'
outFolder= Path(f'{BASE_DIR}/national-dashboards-maplayers')
outputPath = f'{outFolder.parents[1]}/processed/wdpa_protected_areas_public.shp'

In [ ]:
def mbtilesGeneration(data_path: Path, output_path: Union[Path, None] = None,

 update: bool = False) -> Path:
    """
    Simplify geometry of a GeoDataFrame using tippecanoe.
    Parameters
    ----------
    data_path : Path - The path to the GeoDataFrame to simplify.
    output_path : Path - The path to the output GeoDataFrame.
    update : bool, optional - If True, the output GeoDataFrame will be overwritten.
                            The default is False.
    
    Returns
    -------
    Path - The path to the generated mbtiles file.
    """
    try:
        assert data_path.exists(), 'Data path does not exist.'
        if not output_path:
            output_path = data_path.with_suffix('.mbtiles')
        
        if update or not output_path.exists():
            
            if data_path.suffix != '.json':
                CMD = f'mapshaper {data_path} -clean allow-overlaps rewind -o format=geojson {data_path.with_suffix(".json")} force'
                subprocess.run(CMD ,shell=True, check=True)
                data_path = data_path.with_suffix('.json')

            assert data_path.suffix == '.json', 'Data path must be a json file.'
            
            logging.info('Creating mbtiles file...')
            
            subprocess.run(
                f"tippecanoe -zg -f -P -o {output_path} --extend-zooms-if-still-dropping {data_path}",
                shell=True, check=True
                )
        
        return output_path
    
    except Exception as e:
        logging.error(e)
        return 1

In [ ]:
for infile in outFolder.glob('*.shp'):
    logging.info(f'Processing {infile}')
    mbtilesGeneration(infile, update=True)

In [ ]:
#!npm install -g @mapbox/mbview  

#!mbview $outputPath_mbtiles